### Interpolation evaluation

In [1]:
import pandas as pd
import numpy as np
import random
from pathlib import Path
import sqlite3
import itertools

In [2]:
file_name = "data/ALL_COURSES_MOD_TS_BI.csv"
db_name = "results/interpolation.db"

In [3]:
# Read the file of the time series with missing values in a pandas dataframe

time_series = pd.read_csv(file_name, index_col=0)
time_series.index = range(len(time_series))
time_series.columns = range(len(time_series.columns))

In [4]:
# Calcuate the number of missing values

nan_per_week = np.zeros(len(time_series.columns))
nan_ts = time_series.loc[(pd.isna(time_series)).any(axis=1)]
for _, row in nan_ts.iterrows():
    for i, x in enumerate(row):
        if pd.isna(x):
            nan_per_week[i] += 1
perc_nan = sum(nan_per_week)/len(nan_ts)
display(sum(nan_per_week))
display(perc_nan)

55.0

1.5277777777777777

In [5]:
# Drop the time series with missing values
# This will be our ground truth

time_series = time_series.astype('float').dropna()
display(len(time_series))

482

In [6]:
# Create a copy of the time series and artificially insert missing values with the same distribution as the original data

test_time_series = time_series.copy(deep=True)

for i in range(len(test_time_series)):
        # Generate the position to insert Nan with the same distribution as the original data
        pos = random.choices(time_series.columns, nan_per_week/sum(nan_per_week), k=int(perc_nan+random.random()))
        for j in pos:
                test_time_series.iloc[i, j] = np.nan
display(test_time_series.head())

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,33,34
0,11.696970,7.573924,10.076923,4.810995,4.932605,10.262821,5.913889,8.319048,10.054054,4.321731,...,5.927778,5.164021,6.817204,NaN,5.667464,10.478261,6.988095,10.201883,8.382353,7.100928
1,NaN,10.533823,11.124444,10.805533,10.948623,10.054403,11.002451,11.289617,11.154426,10.351446,...,9.368590,11.052455,10.674027,10.250000,8.839912,12.000000,8.095588,9.736218,9.129466,11.646465
2,10.166667,9.773148,10.187908,10.426161,9.448622,11.425461,11.500000,8.217300,8.916779,10.186894,...,11.625000,9.822082,7.870968,5.904762,8.319016,9.052036,11.359765,10.705744,10.009827,6.377207
3,11.666667,11.653846,9.804947,9.261905,8.858953,9.289773,8.682105,NaN,6.998696,7.821769,...,11.186748,8.035377,7.840000,7.626087,10.394886,10.568182,8.000000,8.384310,6.932692,9.491168
4,NaN,7.763702,10.364548,9.724675,9.561748,5.340790,8.508052,7.553977,10.329057,8.074809,...,8.867032,8.749389,7.921690,5.610578,7.517460,4.902174,8.359848,8.742850,7.141026,6.500000


In [7]:
# Function to calculate the mean squared error between 2 time series

def mean_squared_error(y_true, y_pred):

    # Check that the arrays are the same length
    if len(y_true) != len(y_pred):
        raise ValueError("Arrays must have the same length")

    # Calculate the squared errors between the true and predicted values
    squared_errors = (y_true - y_pred) ** 2
    
    # Calculate the mean squared error
    mse = np.mean(squared_errors)

    return mse

In [8]:
# Function used to interpolate the time series with artificially inserted missing
# The type of interpolation changes based on the paramters on input

def interpolation(args):
    
    interpolated_time_series = test_time_series.interpolate(method=args['method'], axis=args['axis'], order=args['order'], limit_area=args['limit_area'], limit_direction=args['limit_direction']).dropna()
    
    mse = []
    for i in interpolated_time_series.index:
        mse.append(mean_squared_error(time_series.loc[i].to_numpy(), interpolated_time_series.loc[i].to_numpy()))
    if len(mse) > 0:
        mse_mean = float(sum(mse)/len(mse))
    else:
        mse_mean = np.nan
    row_dropped = len(time_series)-len(interpolated_time_series)
    return (
        args['method'],
        args['order'],
        args['axis'],
        args['limit_area'],
        args['limit_direction'],
        mse_mean,
        row_dropped
    )

In [9]:
# Combinations of parameters to test
parameters = {
    "method": ['linear', 
               'pad', 'backfill',
               'index', #values is the same
               'nearest', 'zero', 'slinear', 'quadratic', 'cubic', 'barycentric',
               'spline', 'polynomial', #require order
               'krogh', 'piecewise_polynomial', 'pchip', 'akima', 'cubicspline',
               'from_derivatives'
               ],
    "axis": [0, 1],
    "order": [1, 3, 5],
    "limit_area": [None, 'inside'],
    "limit_direction": ['forward', 'backward', 'both']
}

# Query to insert the result of testing in a DB
query_str = '''
                INSERT INTO results(
                    method,
                    poly_spline_order,
                    axis,
                    limit_area,
                    limit_direction,
                    mse,
                    nan_dropped
                ) VALUES (?, ?, ?, ?, ?, ?, ?)
            '''

In [10]:
# Create the DB for the results of interpolation

def init_database(db_name):
    db_path = Path('./' + db_name)
    if db_path.exists():
        db_path.unlink()
    con = sqlite3.connect(db_name)
    cur = con.cursor()
    cur.execute('''
                    CREATE TABLE results(
                        method,
                        poly_spline_order,
                        axis,
                        limit_area,
                        limit_direction,
                        mse,
                        nan_dropped
                    )
                ''')
    return con, cur

In [11]:
# Run a test for each combination of parameters and store the results in a database

keys = list(parameters)
parameters_list = []
# the asterisk (*) symbol in front of the map function is used to unpack the elements of a collection
for values in itertools.product(*map(parameters.get, keys)):
    parameters_list.append(dict(zip(keys, values)))
for p in parameters_list.copy():
    if p["order"] > 1 and p["method"] != "polynomial" and p["method"] != "spline": 
        parameters_list.remove(p)
    else:
        if p["method"] == "pad" and p["limit_direction"] != "forward":
            parameters_list.remove(p)
        if p["method"] == "backfill" and p["limit_direction"] != "backward":
            parameters_list.remove(p)
print(f"Number of test cases for {db_name}: {len(parameters_list)}")    
tuples_list = []
for i in range(len(parameters_list)):
    print(str(i) + "/" + str(len(parameters_list)))
    tuples_list.append(interpolation(parameters_list[i]))
con, cur = init_database(db_name)
cur.executemany(query_str, tuples_list)
con.commit()
con.close()

Number of test cases for results/interpolation.db: 248
0/248
1/248
2/248
3/248
4/248
5/248
6/248
7/248
8/248
9/248
10/248
11/248
12/248
13/248
14/248
15/248
16/248
17/248
18/248
19/248
20/248
21/248
22/248
23/248
24/248
25/248
26/248
27/248
28/248
29/248
30/248
31/248
32/248
33/248
34/248
35/248
36/248
37/248
38/248
39/248
40/248
41/248
42/248
43/248
44/248
45/248
46/248
47/248
48/248
49/248
50/248
51/248
52/248
53/248
54/248
55/248
56/248
57/248
58/248
59/248
60/248
61/248
62/248
63/248
64/248
65/248
66/248
67/248
68/248
69/248
70/248
71/248
72/248
73/248
74/248
75/248
76/248
77/248
78/248
79/248
80/248
81/248
82/248
83/248
84/248
85/248
86/248
87/248
88/248
89/248
90/248
91/248
92/248


c:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\interpolate\_polyint.py:658: RuntimeWarning: divide by zero encountered in divide
  p = np.dot(c,self.yi)/np.sum(c,axis=-1)[...,np.newaxis]


93/248
94/248
95/248
96/248
97/248
98/248
99/248
100/248
101/248
102/248
103/248
104/248
105/248
106/248
107/248
108/248
109/248
110/248
111/248
112/248
113/248
114/248
115/248
116/248
117/248
118/248
119/248
120/248
121/248
122/248
123/248
124/248
125/248
126/248
127/248
128/248
129/248
130/248
131/248
132/248
133/248
134/248
135/248
136/248
137/248
138/248
139/248
140/248
141/248
142/248
143/248
144/248
145/248
146/248
147/248
148/248
149/248
150/248
151/248
152/248
153/248
154/248
155/248
156/248
157/248
158/248
159/248
160/248
161/248
162/248
163/248
164/248
165/248
166/248
167/248
168/248
169/248
170/248
171/248
172/248
173/248
174/248
175/248
176/248


c:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\interpolate\_polyint.py:324: RuntimeWarning: overflow encountered in multiply
  pi = w*pi
c:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\interpolate\_polyint.py:325: RuntimeWarning: invalid value encountered in add
  p += pi[:,np.newaxis] * self.c[k]
c:\Users\david\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\interpolate\_polyint.py:325: RuntimeWarning: invalid value encountered in multiply
  p += pi[:,np.newaxis] * self.c[k]


177/248
178/248
179/248
180/248
181/248
182/248
183/248
184/248
185/248
186/248
187/248
188/248
189/248
190/248
191/248
192/248
193/248
194/248
195/248
196/248
197/248
198/248
199/248
200/248
201/248
202/248
203/248
204/248
205/248
206/248
207/248
208/248
209/248
210/248
211/248
212/248
213/248
214/248
215/248
216/248
217/248
218/248
219/248
220/248
221/248
222/248
223/248
224/248
225/248
226/248
227/248
228/248
229/248
230/248
231/248
232/248
233/248
234/248
235/248
236/248
237/248
238/248
239/248
240/248
241/248
242/248
243/248
244/248
245/248
246/248
247/248
